In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import eigh  # Used for diagonalizing the Hamiltonian

def pauli_matrices():
    """ Return the Pauli matrices sigma_x and sigma_z. """
    sigma_x = np.array([[0, 1], [1, 0]])
    sigma_z = np.array([[1, 0], [0, -1]])
    return sigma_x, sigma_z

def transverse_field_ising_hamiltonian(n, J, h):
    """
    Constructs the Hamiltonian matrix for the transverse field Ising model.
    
    Args:
        n: number of spins (nodes)
        J: interaction strength (between adjacent spins)
        h: transverse field strength
    
    Returns:
        H: Hamiltonian matrix (numpy array of shape (2^n, 2^n))
    """
    # Pauli matrices
    sigma_x, sigma_z = pauli_matrices()

    # Identity matrix
    I = np.eye(2)
    
    # Start with zero Hamiltonian
    H = np.zeros((2**n, 2**n), dtype=complex)
    
    # Interaction part: -J sum (sigma_i^z sigma_{i+1}^z)
    for i in range(n-1):
        term = np.kron(np.eye(2**i), np.kron(sigma_z, np.kron(np.eye(2**(n-i-2)), sigma_z)))
        H -= J * term
    
    # Transverse field part: -h sum sigma_i^x
    for i in range(n):
        term = np.kron(np.eye(2**i), np.kron(sigma_x, np.eye(2**(n-i-1))))
        H -= h * term
    
    return H

def compute_probability_distribution(H):
    """
    Computes the probability distribution for the system using the Hamiltonian.
    
    Args:
        H: Hamiltonian matrix
    
    Returns:
        probabilities: Probability distribution over all possible configurations
    """
    # Compute eigenvalues and eigenvectors
    eigenvalues, eigenvectors = eigh(H)
    
    # Compute the partition function Z
    Z = np.sum(np.exp(-eigenvalues))  # Boltzmann factor summation
    
    # Compute the probability distribution (Boltzmann distribution)
    probabilities = np.exp(-eigenvalues) / Z
    
    return eigenvalues, probabilities

# Example parameters
n = 21  # Number of spins
J = 1.0  # Interaction strength
h = 0.25  # Transverse field strength

# Construct the Hamiltonian
H = transverse_field_ising_hamiltonian(n, J, h)

# Compute the probability distribution
eigenvalues, probabilities = compute_probability_distribution(H)

# Map eigenstates to configurations
# For this simulation, we assume configurations are based on the number of up spins
num_up_spins = np.zeros(2**n, dtype=int)

for idx in range(2**n):
    # Convert index to binary string, count the number of '1's (up spins)
    num_up_spins[idx] = bin(idx).count('1')

# Create bins based on the number of up spins (0 to n up spins)
bins = np.arange(n+1)  # Bins from 0 to n (inclusive)

# Compute the probability distribution over these bins
probabilities_per_bin = np.zeros(n+1)

for i in range(2**n):
    up_spins = num_up_spins[i]
    probabilities_per_bin[up_spins] += probabilities[i]

# Normalize the probabilities to ensure they sum to 1
probabilities_per_bin /= np.sum(probabilities_per_bin)

In [ ]:
# Now, plot the probability distribution over the bins (outside any function)
plt.bar(bins, probabilities_per_bin, width=0.8, color='blue', alpha=0.7)
plt.xlabel('Number of Up Spins')
plt.ylabel('Probability')
plt.title(f"Probability Distribution for Transverse Field Ising Model (n={n}, J={J}, h={h})")
plt.xticks(bins)  # Set x-ticks to correspond to the number of up spins
plt.show()